# Issue OID4VC

The flow logs in with `k.mweene`:`test` and defaults to the **most recently registered birth event** it finds. You can optionally provide your own UUID.

### Get a birth event from OpenCRVS

In [23]:

// Get the latest birth event from OpenCRVS .. 

import { authenticate } from './opencrvs-api.ts'
const token = await authenticate('k.mweene', 'test');

import api from "@opencrvs/toolkit/api";
const { createClient } = api;
const client = createClient('http://localhost:7070/events', `Bearer ${token}`);

const { results } = await client.event.search.query({
  query: {
    type: 'and', clauses: [{ eventType: 'birth' }]
  },
  sort: [{ field: 'createdAt', direction: 'desc' }],
})

// .. or provide a specific event UUID

export const eventId = prompt("Enter UUID of your birth event", results[0]?.id)

// Calculate the current state of the provided event

import events from '@opencrvs/toolkit/events'
const { getCurrentEventState } = events;

const event = await client.event.get.query(eventId)
const configs = await client.event.config.get.query()
const birthConfig = configs.find(({ id }) => id === event.type)!
const birth = getCurrentEventState(event, birthConfig)

Deno.jupyter.md`
\`\`\`json
${JSON.stringify(birth.declaration, null, 2)}
\`\`\`
`



```json
{
  "child.dob": "2025-03-03",
  "child.name": {
    "surname": "Rouvila",
    "firstname": "Pyry",
    "middlename": ""
  },
  "mother.dob": "2002-02-24",
  "mother.name": {
    "surname": "Buffay",
    "firstname": "Phoebe",
    "middlename": "Seni"
  },
  "child.gender": "male",
  "father.reason": "Not available",
  "mother.address": {
    "country": "FAR",
    "addressType": "DOMESTIC",
    "administrativeArea": "ca39e681-e0e8-43a8-af56-c8276800c694"
  },
  "informant.email": "pyry@opencrvs.org",
  "mother.verified": "authenticated",
  "mother.dobUnknown": false,
  "child.placeOfBirth": "HEALTH_FACILITY",
  "informant.relation": "MOTHER",
  "mother.nationality": "FAR",
  "child.birthLocation": "24c68300-1a74-42ff-be26-c3c4e8bbf5f7",
  "mother.query-params": {
    "code": "MTIzNDU2Nzg5Ng==",
    "state": "fetch-on-mount"
  },
  "father.detailsNotAvailable": true,
  "mother.verify-nid-http-fetch": {
    "data": {},
    "loading": false
  },
  "child.nid": "7024925392"
}
```


----

### Issue the VC

1. Provide the details for the credential you want to issue
2. Set-up the HTTP endpoints (`/issue`, `/credential`) that are required for issuance (@TODO: Explain is it the wallet or who that is calling these)
3. 

In [ ]:
const dummyCredentialClaims = {
  sub: "did:example:holder123",
  iss: "http://localhost:4000/issuer",
  nbf: Math.floor(Date.now() / 1000),
  vc: {
    "@context": [
      "https://www.w3.org/2018/credentials/v1",
      "https://www.w3.org/2018/credentials/examples/v1"
    ],
    type: ["VerifiableCredential", "ExampleBadge"],
    credentialSubject: {
      id: "did:example:holder123",
      name: "Jane Doe",
      badge: "Demo PoC Issuer"
    }
  }
};